In [1]:
import pandas as pd
country = "United Kingdom"
year = 2020

In [2]:
df = pd.read_csv("EER_2022_generation.csv")

In [3]:
filt = df["country_name"] == country
filt_year = df['year'] == year

In [6]:
df_target = df[filt][filt_year].set_index("fuel_code").T
df_target

C:\Users\dunca\AppData\Local\Temp/ipykernel_8988/1037320970.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_target = df[filt][filt_year].set_index("fuel_code").T


fuel_code,BIO,GAS,HARDCOAL,HYDRO,NUCLEAR,OTHFOSS,SOLAR,WIND
country_name,United Kingdom,United Kingdom,United Kingdom,United Kingdom,United Kingdom,United Kingdom,United Kingdom,United Kingdom
country_code,GBR,GBR,GBR,GBR,GBR,GBR,GBR,GBR
eu_member,0,0,0,0,0,0,0,0
year,2020,2020,2020,2020,2020,2020,2020,2020
fuel_desc,Bioenergy,Gas,Hard Coal,Hydro,Nuclear,Other Fossil,Solar,Wind
generation_twh,38.1,111.15,5.32,6.69,50.84,8.21,13.32,73.83
share_of_generation_pct,12.39,36.15,1.73,2.18,16.54,2.67,4.33,24.01
yoy_abs_change,0.81,-19.43,-1.57,0.76,-5.34,0.69,0.4,9.5
yoy_pct_change,2.17,-14.88,-22.79,12.82,-9.51,9.18,3.1,14.77


In [7]:
""" CREATING A FOSSILS COLUMN """
df_target["FOSSILS"] = df.sum(numeric_only=True)
df_target

fuel_code,BIO,GAS,HARDCOAL,HYDRO,NUCLEAR,OTHFOSS,SOLAR,WIND,FOSSILS
country_name,United Kingdom,United Kingdom,United Kingdom,United Kingdom,United Kingdom,United Kingdom,United Kingdom,United Kingdom,NaN
country_code,GBR,GBR,GBR,GBR,GBR,GBR,GBR,GBR,NaN
eu_member,0,0,0,0,0,0,0,0,8632.00
year,2020,2020,2020,2020,2020,2020,2020,2020,23158263.00
fuel_desc,Bioenergy,Gas,Hard Coal,Hydro,Nuclear,Other Fossil,Solar,Wind,NaN
generation_twh,38.1,111.15,5.32,6.69,50.84,8.21,13.32,73.83,115726.10
share_of_generation_pct,12.39,36.15,1.73,2.18,16.54,2.67,4.33,24.01,115600.24
yoy_abs_change,0.81,-19.43,-1.57,0.76,-5.34,0.69,0.4,9.5,772.66
yoy_pct_change,2.17,-14.88,-22.79,12.82,-9.51,9.18,3.1,14.77,103592.12


In [ ]:
emojis = {
    'dinosaur'
}